In [10]:
from datascience import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn import tree
from sklearn import metrics
import graphviz

from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import re
import numpy as np
import random

In [12]:
# Load data set
data = Table.read_table('navya_data/lab7_data_nonASCII_removed.csv', sep='\t')
# data = pd.DataFrame.from_csv('navya_data/lab7_data.txt', sep='\t')
data

# d = pd.read_csv(open('navya_data/lab7_data.txt', 'rU'), encoding='utf', engine='c')

text,tagger1,tagger2
RT @Salon: Tax reform for the rich: Trumps plan abandons ...,neu,neu
RT @calebecarma: There's video of Trump shooting paper t ...,neu,neu
"RT @IrisRimon: The NRA gave Donald Trump $30M, he told t ...",neu,neu
RT @KeithOlbermann: NEW VIDEO: The metaphor of Trump thr ...,neu,neu
RT @ProudResister: Trump called #StephenPaddock Pure Evi ...,neu,neu
RT @latimes: Another Trump administration official is un ...,neu,neu
"RT @NBCPolitics: James Mattis, going against Trump campa ...",neu,neu
RT @2020fight: Fun fact. Trump just completed his 16th t ...,neu,neu
"Mark Cuban hits Trump on tax returns, says he is 'consid ...",neu,neu
RT @thehill: Trump tosses paper towels into crowd while ...,neu,neu


In [ ]:
# Separate irrelevant and relevant tweets data
### TODO: complete the where clauses. You may use multiple of them
data_irrelevant = data.where()
data_relevant = data.where()
### END

In [ ]:
# Randomly select NUM_TRAIN relevant and NUM_TRAIN irrelevant as training data, total NUM_TRAIN*2
# NUM_TEST relevant and NUM_TEST irrelevant as testing data, total NUM_TEST*2

### TODO: Assign proper numbers to NUM_TRAIN and NUM_TEST
NUM_TRAIN = None
NUM_TEST = None
### END

assert NUM_TRAIN + NUM_TEST <= data_irrelevant.num_rows
assert NUM_TRAIN + NUM_TEST <= data_relevant.num_rows

relevant_rows = list(range(data_relevant.num_rows))
random.shuffle(relevant_rows)
relevant_rows_train = relevant_rows[:NUM_TRAIN]
relevant_rows_test = relevant_rows[-NUM_TEST:]

irrelevant_rows = list(range(data_irrelevant.num_rows))
random.shuffle(irrelevant_rows)
irrelevant_rows_train = irrelevant_rows[:NUM_TRAIN]
irrelevant_rows_test = irrelevant_rows[-NUM_TEST:]

In [ ]:
# Construct actual train and test datasets: X_train, y_train, X_test, y_test

X_train = list(data_relevant.take(relevant_rows_train)['text']) + \
    list(data_irrelevant.take(irrelevant_rows_train)['text'])
y_train = [1] * NUM_TRAIN + [0] * NUM_TRAIN

print('X_train size:', len(X_train))
print('X_train first 5 elements:', X_train[:5])
print('\ny_train size:', len(y_train))
print('y_train:', y_train)

X_test = list(data_relevant.take(relevant_rows_test)['text']) + \
    list(data_irrelevant.take(irrelevant_rows_test)['text'])
y_test = [1] * NUM_TEST + [0] * NUM_TEST

print('\nX_test size', len(X_test))
print('y_test size', len(y_test))

In [ ]:
# Extract bag of words features
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
print(X_train_counts.shape)

In [ ]:
# Create and fit (train) the classifier
clf = tree.DecisionTreeClassifier()
clf.fit(X_train_counts, y_train)

In [ ]:
# Evaluate on test set
X_test_counts = count_vect.transform(X_test) # Use transform instead of fit_transform
predicted = clf.predict(X_test_counts)

In [ ]:
# Simple report
np.mean(predicted == y_test)

In [ ]:
# Detailed report
print(metrics.classification_report(y_test, predicted))

In [ ]:
# Confusion matrix
metrics.confusion_matrix(y_test, predicted)

In [ ]:
# Alternative method using Pipeline
# create and fit (train)
text_clf = Pipeline([('vect', CountVectorizer()), 
                     ('clf', tree.DecisionTreeClassifier())])
text_clf.fit(X_train, y_train)

In [ ]:
# Evaluate on test set
predicted = text_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted))

In [ ]:
# Visualize
dot_data = tree.export_graphviz(clf, out_file=None, feature_names=count_vect.get_feature_names())

In [ ]:
# Inline plot
graph = graphviz.Source(dot_data)
graph

In [ ]:
# Save to pdf
graph.render('tree')